In [197]:
import numpy as np
import pandas as pd

import datetime
import hashlib
import json

import random


In [198]:
def getID(tt,task):
    string=tt.__str__()+task['op']+task['node']
    m = hashlib.sha1()
    m.update(string.encode('utf-8'))
    return(m.hexdigest())


In [199]:
def createTask(op,node):
    return({'op':op,'node':node})

In [200]:
def createJob(task):
    tt=datetime.datetime.now()
    return({'id':getID(tt,task),'creation_time':tt,'task':tsk})

In [49]:
tsk=createTask('CNCT','0xf08ce459930dc561ceaa')
createJob(tsk)

{'id': 'db9d5cef8ddc1c8a4d0c7c2b9b72d103881bd9ad',
 'creation_time': datetime.datetime(2023, 12, 14, 18, 47, 43, 247554),
 'task': {'op': 'CNCT', 'node': '0xf08ce459930dc561ceaa'}}

In [53]:
dd={'c':3,'d':2,'a':5,'b':4,'e':1}

In [54]:
sorted(dd)

['a', 'b', 'c', 'd', 'e']

In [201]:
#################################################################################################################
# Delegator node events:
#        creation -  when a node is added to thew priority list (or readded after being deleted)
#        connection -  when a worker is assigned a 'CNCT' job to the node and the job is completed
#        pong - when a node appears in the list of neighbors returned from a 'CNCT' job for a different node
#################################################################################################################

class Node:

    def __init__(self,nodeDoc):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.id=nodeDoc['id']
        self.doc=nodeDoc
        self.creation=ut
        self.last_connect=None
        self.last_pong=ut
        self.priority=ut
        return
    
    
    def connect(self):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.last_connect=ut
        self.priority=ut
        return
        
    def pong(self):
        tt=datetime.datetime.now()
        ut=datetime.datetime.timestamp(tt)
        self.last_pong=ut
        self.priority=0.7*ut+0.3*self.priority  # (ut-self.priority)*0.7+self.priority
        return
    
    def __str__(self):
        return(json.dumps(self.doc)+'--'+self.priority.__str__()+'--'+self.last_pong.__str__()+'--'+self.last_connect.__str__())
    
    def __json__(self):
        return(self.doc)

In [104]:
class NodeList:
    
    def __init__(self,len):
        self.nodeList={}
        self.len=MAX_LENGTH
        return
    
    def insert(node):
        self.nodeList[node.id]=node
        return
    
    def remove(nid):
        del(self.nodeList[nid])
        
    def updateNode(nid,update):
        if update='CNCT':
            self.nodeList[node.id].connect()
        elif update='PONG':
            self.nodeList[node.id].pong()
        else:
            raise Exception("Unknown update")
        return

    def pull(self,end):
        tmp={node.priority:node.id for node in self.nodeList}
        if end='head':
            nid=self.nodeList[min(self.nodeList)]
        elif end='tail':
            nid=self.nodeList[max(self.nodeList)]
        node=self.nodeList[nid]
        self.removeNode(nid)
        return(node)
        
    def showPriority(self):
        tmp={node.priority:node.id for node in self.nodeList}
        return(tmp)


In [114]:
ll={0.1:2,0.2:3,0.4:4,0.7:5,0.9:1}
print(ll)
del(ll[0.4])
print(ll)

{0.1: 2, 0.2: 3, 0.4: 4, 0.7: 5, 0.9: 1}
{0.1: 2, 0.2: 3, 0.7: 5, 0.9: 1}


In [115]:
max(ll)

0.9

In [251]:
#################################################################################################################
#   Generating a random network
#################################################################################################################

def randomId():
    tt=random.randint(0,10000)
    string=tt.__str__()
    m = hashlib.sha1()
    m.update(string.encode('utf-8'))
    return('0x'+m.hexdigest()[:20])

def randomCity():
    Cities=['Kabul','Herat','Mazar-i-Sharif','Kandahar','Jalalabad','Lashkargah','Kunduz','Taloqan','Puli Khumri',\
     'Sheberghan','Zaranj','Maymana','Ghazni','Khost','Charikar','Fayzabad','Tarinkot','Gardez']
    return(random.choice(Cities))

def generateNode():
    return({'id':randomId(),'loc':randomCity(),'neigh':[]})


def generateNeigh(Vid):
    nn=random.randint(0,min([20,len(Vid)]))
    neigh=set([random.choice(Vid) for ii in range(nn)])
    return(neigh)

def generateNetwork(nsize):
    V=[generateNode() for ii in range(nsize)]
    Vp={nn['id']:nn for nn in V}
    Vid=[nn['id'] for nn in V]
    for nn in V:
        rneigh=generateNeigh(Vid)-set([nn['id']])
        nn['neigh']=list(rneigh|set(nn['neigh']))
        for ii in rneigh:
            if not nn['id'] in Vp[ii]['neigh']:
                Vp[ii]['neigh'].append(nn['id'])
    return(Vp)


In [260]:
nwk=generateNetwork(10000)
{ii:nwk[ii] for ii in list(nwk.keys())[3448:3453]}
#nwk

{'0x045d84a9fadcd7ae5582': {'id': '0x045d84a9fadcd7ae5582',
  'loc': 'Charikar',
  'neigh': ['0xa5cc040dbb73e42ac7a1',
   '0xf25cbb01e86180748a0f',
   '0xa2bc9c82e25301900618',
   '0x6337fd63957907483264',
   '0x47a78cc892e29d89a7ab',
   '0x78b6e7d58093116dc475',
   '0xf1191008e7f0422a574c',
   '0x8b69f46dcd8c17b4fa15',
   '0x6940cc8da791bf12d7c8',
   '0xe4fec7c72e63a47d1de6',
   '0x2c2c56a113fde28bd291',
   '0xc5946a06ef091106692f',
   '0x12d9b057114a5b072df2',
   '0x52dc2eec697d9b81d823',
   '0x92f2fd99879b0c2466ab',
   '0x2a94b50a655ce6b38f78',
   '0x0da8cbc894c3e556a242',
   '0xd0bb2479acb9c85f83df',
   '0x93acc66a6e5f774385d4',
   '0x28e238f17a59ae02e306',
   '0xfc5dde654b1d78f421f8',
   '0xc8589da1ade1fe6e8cda']},
 '0xf8afd0c8400233e0ab7a': {'id': '0xf8afd0c8400233e0ab7a',
  'loc': 'Charikar',
  'neigh': ['0x2da8dade355cc4b0b359',
   '0x2a5a8b131bc31e96ca6f',
   '0xb4581a839e2e5344ac0c',
   '0x7a4375a1fed653e13860',
   '0x80b7f06dd0f122cb610c',
   '0xdf93151d1f40e8349efc',
   '0x